In [ ]:
from fenics import *
import numpy as np
from braininversion.IOHandling import (read_mesh_from_h5, write_to_xdmf, 
                                       xdmf_to_unstructuredGrid, read_xdmf_timeseries)
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                                           plot_pressures_and_forces_cross_section,
                                           extract_cross_section, style_dict)
import matplotlib.pyplot as plt

In [ ]:
filename_mesh = "../meshes/ideal_brain_subdomains.xdmf" 
filename_results = "../results/ideal_brain_biot_NS_checkp.xdmf"
T = 3.0
num_steps = 100
mmHg2Pa = 132.32

dt = T/num_steps
times = np.linspace(dt, T, num_steps)
infile_mesh = XDMFFile(filename_mesh)
mesh = Mesh()
infile_mesh.read(mesh)

In [ ]:
V = VectorFunctionSpace(mesh, "CG", 2)
W = FunctionSpace(mesh, "CG", 1)
names = {"pF":W, "pP":W, "phi":W,
         #"d":V, , "u":V
        }
infile_results = XDMFFile(filename_results)
results = {n:[] for n in names}
for n, space in names.items():
    for i in range(num_steps):
        f = Function(space)
        infile_results.read_checkpoint(f, n, i)
        results[n].append(f)

In [ ]:
pP_series = results["pP"]
pF_series = results["pF"]
phi_series = results["phi"]


ventricle_probe = [Point(0.0, 0.0)]
sas_probe = [Point(0.0, 0.11)]
parenchyma_probe = [Point(0.0, 0.05)]


pF_sas = extract_cross_section(pF_series, sas_probe).flatten()/mmHg2Pa
pF_ventricle = extract_cross_section(pF_series, ventricle_probe).flatten()/mmHg2Pa
pP_parenchyma = extract_cross_section(pP_series, parenchyma_probe).flatten()/mmHg2Pa
phi_parenchyma = extract_cross_section(phi_series, parenchyma_probe).flatten()/mmHg2Pa


plt.figure(figsize=(10,8))
plt.plot(times, pF_ventricle, label="ventricle")
plt.plot(times, pF_sas, label="SAS")
#plt.plot(times, pP_parenchyma, label="parenchyma fluid")
plt.plot(times, phi_parenchyma, label="parenchyma tot")

plt.legend()
plt.grid()
plt.xlabel("t [s]")
plt.ylabel("p in mmHg")